In [72]:
%matplotlib inline
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from lsst.cwfs.tools import ZernikeEval, ZernikeGrad
import matplotlib.colors as colors
from decimal import Decimal
mpl.rc('image', cmap='jet')

In [73]:
x1d = np.linspace(-1,1,201)
x, y = np.meshgrid(x1d, x1d)
r = np.sqrt(x**2 + y**2)
idx = r>1
x[idx] = np.nan
y[idx] = np.nan
r[idx] = np.nan

maxCP = 100.
numberCP = 10  #+++++++++++++++++++++++++++++++++++++++++++++

# Define the array with values of CP
CP = np.linspace(10.,maxCP,10)  #+++++++++++++


In [74]:
def GetZernikes(CP,numberCP):
    DOF = np.zeros(10)
    Field = 32
    DOF[numberCP-1] = CP

    dataRead = np.loadtxt('./ZvsCP/senM_35_19_50.txt',comments="#")
    dataRead = dataRead.reshape(35,19,50)
    Zs = np.zeros(19)         

    # Calculate the Zernike amplitudes.
    for i in range(0,len(Zs)):
        Zs[i] = 0.0
        for j in range(0,len(DOF)):
            Zs[i] = Zs[i] + DOF[j]*dataRead[Field-1,i,j]
    return Zs

In [77]:

ZCoef = [GetZernikes(i,numberCP) for i in CP]
zc = np.zeros([10, 22])

for index in range(3,22):
    zc[:,index] = [ZCoef[i][index-3] for i in range(0,len(CP))]

# Define the wavefront and set it to 0.
w = np.zeros([10,201,201])
dwdx = np.zeros([10,201,201])
dwdy = np.zeros([10,201,201])
rmsSpotSize = np.zeros(10)

thetaCone = np.zeros(10)
thetaDist = w


for i in range(10):
    w[i,:,:] = ZernikeEval(zc[i,:], x, y)
    dwdx[i,:,:] = ZernikeGrad(zc[i,:], x, y, 'dx')
    dwdy[i,:,:] = ZernikeGrad(zc[i,:], x, y, 'dy')

    xr = dwdx[i,:,:].reshape(-1,1)
    yr = dwdy[i,:,:].reshape(-1,1)

    idx = ~np.isnan(xr)
    xr = xr[idx]
    yr = yr[idx]
    rmsSpotSize[i] = np.sqrt(np.sum(xr**2 + yr**2))
    
    
    thetaDist[i,:,:] = 1.0E6*np.arccos(1./np.sqrt((dwdx[i,:,:]*1.2E-7)**2+(dwdy[i,:,:]*1.2E-7)**2+1.))
    

    xr = dwdx[i,:,:].reshape(-1,1)
    yr = dwdy[i,:,:].reshape(-1,1)
    idx = ~np.isnan(xr)
    xr = xr[idx]
    yr = yr[idx]  
    
    theta = np.arccos(1./np.sqrt((xr*1.2E-7)**2+(yr*1.2E-7)**2+1.))
    # Instead, we calculate sin(theta) and the use the approximation that sin(theta) = theta for small angles
    theta = np.sqrt((xr*1.2E-7)**2+(yr*1.2E-7)**2)/np.sqrt((xr*1.2E-7)**2+(yr*1.2E-7)**2+1.)    
    

/Users/junyin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in arccos


1.1257414656076032
